In [4]:
!pip install folium

In [5]:
# Step 1: Install required libraries if you haven't already
!pip install matplotlib scipy Pillow simplekml

# Step 2: Import all necessary modules
import logging
import numpy as np
import simplekml
from datetime import datetime
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
import os
import zipfile
from PIL import Image

# Imports from astra-simulator
from astra.simulator import flight
from astra.weather import forecastEnvironment

logging.basicConfig(level=logging.INFO)

def run_trajectory_heatmap_simulation(num_runs=10):
    """
    Runs multiple flight simulations and generates a KMZ file with a heatmap,
    labeled landing points, and labeled flight trajectories.
    """
    # --- Simulation Parameters ---
    launch_datetime = datetime(2025, 7, 23, 12, 0)
    launchLat, launchLon, launchElev = (50.903824, -1.63697, 114.0)

    print("Preparing weather environment...")
    env = forecastEnvironment(launchLat, launchLon, launchElev, launch_datetime)
    print("Weather environment ready.")

    all_profiles = []
    print(f"Running {num_runs} simulations...")

    for i in range(num_runs):
        randomized_nozzle_lift = 1.0 + (np.random.rand() - 0.5) * 0.4
        print(f"\rRun {i + 1}/{num_runs}", end="")

        sim = flight(
            environment=env,
            balloonGasType='Helium',
            balloonModel='TA100',
            nozzleLift=randomized_nozzle_lift,
            payloadTrainWeight=0.38,
            parachuteModel='SPH36',
            trainEquivSphereDiam=0.1,
            maxFlightTime=18000,
            numberOfSimRuns=1
        )
        sim._preflight(env.dateAndTime)
        result, _ = sim.fly(flightNumber=0, launchDateTime=env.dateAndTime, runPreflight=False)

        if result:
            all_profiles.append(result)

    print("\nAll simulations complete.")

    if not all_profiles:
        print("No simulation data was collected. Cannot generate KML.")
        return

    # --- Heatmap Generation ---
    landing_points = [(p.longitudeProfile[-1], p.latitudeProfile[-1]) for p in all_profiles]
    print("Generating high-resolution heatmap image...")
    lons, lats = zip(*landing_points)
    margin = 0.05
    north, south = max(lats) + margin, min(lats) - margin
    east, west = max(lons) + margin, min(lons) - margin

    try:
        kde = gaussian_kde([lons, lats])
        resolution = 400
        grid_lon, grid_lat = np.mgrid[west:east:complex(0, resolution), south:north:complex(0, resolution)]
        heatmap_values = kde.evaluate([grid_lon.ravel(), grid_lat.ravel()])
    except np.linalg.LinAlgError:
        print("\nWarning: Could not generate a precise heatmap. Creating an empty overlay.")
        resolution = 400
        heatmap_values = np.zeros((resolution, resolution))

    heatmap_img = heatmap_values.reshape((resolution, resolution))
    fig = plt.figure(figsize=(4, 4), dpi=100)
    ax = fig.add_axes([0, 0, 1, 1])
    ax.axis('off')
    ax.imshow(np.rot90(heatmap_img), cmap=plt.cm.hot_r, alpha=0.5)
    temp_heatmap_filename = "temp_heatmap.png"
    fig.savefig(temp_heatmap_filename, transparent=True, pad_inches=0)
    plt.close(fig)
    img = Image.open(temp_heatmap_filename).convert("RGBA")
    datas = img.getdata()
    newData = []
    for item in datas:
        if item[0] > 240 and item[1] > 240 and item[2] > 240:
            newData.append((255, 255, 255, 0))
        else:
            newData.append(item)
    img.putdata(newData)
    heatmap_filename = "landing_heatmap_final.png"
    img.save(heatmap_filename, "PNG")
    os.remove(temp_heatmap_filename)

    # --- KML and KMZ Creation ---
    print("Creating KML and packaging into KMZ...")
    kml = simplekml.Kml()
    kml.document.name = "Flight Trajectories and Landing Zone"

    # 1. Add the Heatmap
    ground = kml.newgroundoverlay(name='Landing Zone Heatmap')
    ground.icon.href = heatmap_filename
    ground.latlonbox.north, ground.latlonbox.south = north, south
    ground.latlonbox.east, ground.latlonbox.west = east, west

    # 2. Add Labeled Trajectories and Landing Points
    for i, profile in enumerate(all_profiles):
        # Create a folder for each flight path to keep it organized
        folder = kml.newfolder(name=f"Flight #{i + 1}")
        
        # Add the trajectory line
        linestring = folder.newlinestring(name="Flight Path")
        coords = list(zip(profile.longitudeProfile, profile.latitudeProfile, profile.altitudeProfile))
        linestring.coords = coords
        linestring.altitudemode = simplekml.AltitudeMode.absolute
        linestring.extrude = 0
        linestring.style.linestyle.width = 2
        
        # THE FIX: Change color to blue and set transparency
        alpha_hex = format(180, '02x') # ~70% opaque
        linestring.style.linestyle.color = simplekml.Color.changealpha(alpha_hex, simplekml.Color.blue)

        # Add the corresponding landing point marker
        landing_lon, landing_lat = profile.longitudeProfile[-1], profile.latitudeProfile[-1]
        folder.newpoint(name=f"Landing Site", coords=[(landing_lon, landing_lat)])

    # Manually create the KMZ archive
    kml_filename = "doc.kml"
    kml.save(kml_filename)
    output_filename_kmz = "flight_trajectories.kmz"
    with zipfile.ZipFile(output_filename_kmz, 'w', zipfile.ZIP_DEFLATED) as kmz:
        kmz.write(kml_filename)
        kmz.write(heatmap_filename)
    os.remove(kml_filename)
    os.remove(heatmap_filename)

    print(f"\nSuccess! Open {output_filename_kmz} in Google Earth to see the final visualization.")

# Run the full process
run_trajectory_heatmap_simulation(num_runs=10)

ModuleNotFoundError: No module named 'astra'